In [37]:
import pymysql
import getpass as gp
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [38]:
# conn1 = pymysql.connect(host='127.0.0.1', user='root', password = input(), db='employees', charset='utf8mb4')
# conn2 = pymysql.connect(host='127.0.0.1', user='reader', password = input(), db='employees', charset='utf8mb4')

In [39]:
# 계정 정보 사전
accounts = {
    'root': {'host': '127.0.0.1', 'db': 'employees'},
    'reader': {'host': '127.0.0.1', 'db': 'employees'},
    'writer': {'host': '127.0.0.1', 'db': 'employees'},
    # 여기에 계정 추가 가능
}

def connect_to_db():
    user = input("ID: ").strip()
    if user not in accounts:
        raise ValueError("등록되지 않은 사용자 ID")

    password = gp.getpass("비밀번호: ")
    info = accounts[user]

    conn = pymysql.connect(
        host=info['host'],
        user=user,
        password=password,
        db=info['db'],
        charset='utf8mb4'
    )
    return conn

def q(query):
    with conn.cursor() as cursor: # 커서 생성 with 구문을 사용해서 자동으로 close 하므로 메모리 누수를 방지
        cursor.execute(query)
        first = query.strip().split()[0].lower()
        if first in ['select', 'show', 'describe', 'desc', 'explain']:
            df = pd.read_sql(query, conn)
            display(df)
        else:
            conn.commit()
            print("Query OK.")


In [40]:
conn = connect_to_db()
q("SELECT current_role();")

,current_role()
0,NONE


In [41]:
q("SELECT user, host, account_locked FROM mysql.user;")

,user,host,account_locked
0,role_emp_read,%,Y
1,role_emp_write,%,Y
2,reader,127.0.0.1,N
3,writer,127.0.0.1,N
4,edward,192.168.0.28,N
5,mysql.infoschema,localhost,Y
6,mysql.session,localhost,Y
7,mysql.sys,localhost,Y
8,root,localhost,N


In [42]:
q("SELECT user, host, account_locked FROM mysql.user WHERE account_locked = 'Y';")

,user,host,account_locked
0,role_emp_read,%,Y
1,role_emp_write,%,Y
2,mysql.infoschema,localhost,Y
3,mysql.session,localhost,Y
4,mysql.sys,localhost,Y


In [43]:
q("SELECT user, host, account_locked FROM mysql.user WHERE account_locked = 'N';")

,user,host,account_locked
0,reader,127.0.0.1,N
1,writer,127.0.0.1,N
2,edward,192.168.0.28,N
3,root,localhost,N


| 구문                 | 재실행 시 영향            | 설명                                                            |
| ------------------ | ------------------- | ------------------------------------------------------------- |
| `INSERT INTO`      | ❗**데이터 중복 삽입**      | 같은 레코드 계속 추가됨                                                 |
| `UPDATE`           | ❗**데이터 덮어쓰기**       | 조건 잘못 걸면 의도치 않게 전체 업데이트                                       |
| `DELETE`           | ❗**데이터 삭제**         | 조건 실수하면 전체 삭제 가능                                              |
| `TRUNCATE TABLE`   | ❗**모든 데이터 삭제**      | 되돌릴 수 없음 (DML 아님)                                             |
| `DROP TABLE`       | ❗**테이블 자체 삭제**      | 데이터+구조 완전 삭제됨                                                 |
| `DROP DATABASE`    | ❗**DB 전체 삭제**       | 되돌릴 수 없음                                                      |
| `CREATE TABLE`     | ⚠️ **에러 또는 덮어씀**    | `IF NOT EXISTS` 안 쓰면 에러. `LIKE`, `AS SELECT` 형태는 구조 복제 가능성 주의 |
| `ALTER TABLE`      | ⚠️ **구조 덮어쓰기/변경**   | 컬럼 추가/변경 시 기존 데이터 손상 위험                                       |
| `LOAD DATA INFILE` | ❗**대량 삽입**          | 실수로 중복 데이터 대량 삽입 가능                                           |
| `REPLACE INTO`     | ❗**기존 데이터 삭제 후 삽입** | `PRIMARY KEY` 충돌 시 기존 데이터 삭제됨                                 |

위 구문들은 한번 사용하면 주석 처리해서 중복사용 안되게 조심해서 실행

In [ ]:
q("""
CREATE TABLE IF NOT EXISTS users (
    id INT AUTO_INCREMENT PRIMARY KEY,
    name VARCHAR(50),
    age INT
);
""")
# q("INSERT INTO users (name, age) VALUES ('Alice', 25), ('Bob', 30);")
q("SELECT * FROM users;")


Query OK.
Query OK.


,id,name,age
0,1,Alice,25
1,2,Bob,30
2,3,Alice,25
3,4,Bob,30
4,5,Alice,25
5,6,Bob,30


In [45]:
q('SELECT DATABASE();')

,DATABASE()
0,employees


In [46]:
q("SHOW TABLES;")

,Tables_in_employees
0,departments
1,dept_emp
2,dept_manager
3,employee_docs
4,employee_name
5,employees
6,employees_comp4k
7,employees_comp8k
8,salaries
9,tb_dual


In [47]:
q("DESCRIBE users;")

,Field,Type,Null,Key,Default,Extra
0,id,int,NO,PRI,None,auto_increment
1,name,varchar(50),YES,,None,
2,age,int,YES,,None,


In [48]:
q("SELECT * FROM users;")

,id,name,age
0,1,Alice,25
1,2,Bob,30
2,3,Alice,25
3,4,Bob,30
4,5,Alice,25
5,6,Bob,30


In [49]:
q("DROP TABLE IF EXISTS users;")

Query OK.
